## Genetic Algorithm for Neural Network Weight Optimization

### Pseudo Code

### Input
- **X_train**, **y_train**: Training data
- **window_size**: Number of past observations (input features)
- **hidden_size**: Number of neurons in the hidden layer
- **population_size**: Number of chromosomes (individuals)
- **n_generations**: Number of generations (iterations)
- **param_size**: Total number of parameters to optimize (weights + biases)

### Representation
- **Real-coded representation** is used:
  - Each chromosome is a real-valued vector directly corresponding to the neural network's weights and biases.
  - **No binary encoding or decoding** is required.

### Initialization
- Randomly create `population_size` individuals, each being a real-valued vector of length `param_size`.
- For each individual, evaluate its fitness using Mean Squared Error (MSE) loss on (X_train, y_train).

### Evolution Process

#### For each generation from 1 to n_generations:

1. **Selection**
    - Select the top 50% individuals with the lowest fitness scores (elitism strategy).

2. **Crossover**
    - While the size of the next population is less than `population_size`:
        - Randomly select two parents from the selected individuals.
        - Independently sample a crossover probability from a Beta(2,5) distribution.
        - If a random number is less than the sampled crossover probability:
            - Perform one-point crossover to generate two children.
        - Else:
            - Children are exact copies of the parents.

3. **Mutation**
    - For each generated child:
        - Independently sample a mutation probability from a Normal(μ=0.1, σ=0.05) distribution, clipped between [0.01, 0.5].
        - For each gene (weight or bias) in the child:
            - If a random number is less than the mutation probability:
                - Apply additive Gaussian perturbation (mean=0, std=0.1) to the gene.

4. **Update**
    - Replace the current population with the newly generated individuals.
    - Evaluate the fitness scores for all individuals in the new population.

5. **Track Best**
    - Update and store the best individual (the one with the lowest fitness) found so far.

#### End of Evolution

### Output
- The best-found parameters (weights and biases of the neural network).

### Testing
- Forward-pass **X_test** through the neural network using the best-found parameters.
- Calculate and report:
  - Test Mean Squared Error (MSE)
  - Test Mean Absolute Error (MAE)

In [37]:
!pip install numpy
!pip install yfinance

In [24]:
import numpy as np
import random
import yfinance as yf

# --- Load data ---
ticker = yf.Ticker("005930.KS")
price_data = ticker.history(period="1d", interval="1m")
n = 200
recent = price_data.tail(n)
prices = recent['Close'].values

# --- Hyperparameters ---
window_size = 5
hidden_size = 10
param_size = window_size * hidden_size + hidden_size + hidden_size + 1

# --- Prepare dataset (X: past window_size prices, y: next price) ---
X = []
y = []
for i in range(len(prices) - window_size):
    X.append(prices[i:i+window_size])
    y.append(prices[i+window_size])
X = np.array(X)
y = np.array(y)

# --- Train/Test Split ---
split_idx = int(0.7 * len(X))
X_train = X[:split_idx]
y_train = y[:split_idx]
X_test = X[split_idx:]
y_test = y[split_idx:]

# --- Define MLP Model ---
def forward(x, params, input_dim, hidden_dim):
    W1 = params[:input_dim * hidden_dim].reshape(input_dim, hidden_dim)
    b1 = params[input_dim * hidden_dim : input_dim * hidden_dim + hidden_dim]
    W2 = params[input_dim * hidden_dim + hidden_dim : input_dim * hidden_dim + hidden_dim + hidden_dim].reshape(hidden_dim, 1)
    b2 = params[-1]

    hidden = np.dot(x, W1) + b1
    hidden = np.maximum(hidden, 0)  # ReLU
    output = np.dot(hidden, W2) + b2
    return output.squeeze()

# --- Loss Function (MSE) ---
def mse_loss(params, X, y, input_dim, hidden_dim):
    preds = np.array([forward(x, params, input_dim, hidden_dim) for x in X])
    return np.mean((preds - y)**2)

# --- Genetic Algorithm Class ---
class GA:
    def __init__(self, population_size, dim, X, y, n_generations, input_dim, hidden_dim):
        self.population_size = population_size
        self.dim = dim
        self.X = X
        self.y = y
        self.n_generations = n_generations
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.population = [np.random.randn(dim) for _ in range(population_size)]
        self.fitness = [self.evaluate(ind) for ind in self.population]

    def evaluate(self, individual):
        return mse_loss(individual, self.X, self.y, self.input_dim, self.hidden_dim)

    def selection(self):
        idx = np.argsort(self.fitness)
        return [self.population[i] for i in idx[:self.population_size//2]]

    def crossover(self, parent1, parent2):
        crossover_prob = np.random.beta(2, 5)  # Beta distribution
        if random.random() < crossover_prob:
            point = random.randint(1, self.dim-1)
            child1 = np.concatenate((parent1[:point], parent2[point:]))
            child2 = np.concatenate((parent2[:point], parent1[point:]))
            return child1, child2
        else:
            return parent1.copy(), parent2.copy()

    def mutation(self, individual):
        mutation_prob = np.clip(np.random.normal(0.1, 0.05), 0.01, 0.5)  # Normal Distribution
        for i in range(self.dim):
            if random.random() < mutation_prob:
                individual[i] += np.random.normal(0, 0.1)
        return individual

    def evolve(self):
        for gen in range(self.n_generations):
            selected = self.selection()
            next_population = selected.copy()

            while len(next_population) < self.population_size:
                parents = random.sample(selected, 2)
                child1, child2 = self.crossover(parents[0], parents[1])
                next_population.append(self.mutation(child1))
                if len(next_population) < self.population_size:
                    next_population.append(self.mutation(child2))

            self.population = next_population
            self.fitness = [self.evaluate(ind) for ind in self.population]
            
            if gen % 10 == 0:
                print(f"Generation {gen}, Best Loss: {min(self.fitness):.6f}")

        best_idx = np.argmin(self.fitness)
        return self.population[best_idx]

# --- Run GA ---
print("GA")
ga = GA(population_size=30, dim=param_size, X=X_train, y=y_train,
        n_generations=100, input_dim=window_size, hidden_dim=hidden_size)
best_params = ga.evolve()

# --- Train/Test Time Range ---
train_timestamps = recent.index[window_size:split_idx+window_size]
test_timestamps = recent.index[split_idx+window_size:]

train_start_time = train_timestamps[0]
train_end_time = train_timestamps[-1]
test_start_time = test_timestamps[0]
test_end_time = test_timestamps[-1]

print("\nDATA PERIODS")
print(f"Train set: {train_start_time.strftime('%H:%M')} ~ {train_end_time.strftime('%H:%M')}")
print(f"Test set: {test_start_time.strftime('%H:%M')} ~ {test_end_time.strftime('%H:%M')}")


# --- Test Evaluation with timestamp ---
predictions = np.array([forward(x, best_params, window_size, hidden_size) for x in X_test])

test_timestamps = recent.index[split_idx+window_size:]

print("\nTEST PREDICTION")
print(f"{'Timestamp':<25} {'Real':<15} {'Prediction':<15}")
print("-"*55)
for ts, real, pred in zip(test_timestamps, y_test, predictions):
    print(f"{ts} {real:<15.6f} {pred:<15.6f}")

mse_test = np.mean((predictions - y_test)**2)
absolute_errors = np.abs(predictions - y_test)
mean_absolute_error = np.mean(absolute_errors)

print("\nTest MSE:", round(mse_test, 6))
print("Test Mean Absolute Error:", round(mean_absolute_error, 6))

GA
Generation 0, Best Loss: 5689942.023401
Generation 10, Best Loss: 19291.138665
Generation 20, Best Loss: 8440.889432
Generation 30, Best Loss: 5288.763019
Generation 40, Best Loss: 5209.068426
Generation 50, Best Loss: 5167.794624
Generation 60, Best Loss: 5162.149554
Generation 70, Best Loss: 5059.708639
Generation 80, Best Loss: 5059.429966
Generation 90, Best Loss: 5059.024253

DATA PERIODS
Train set: 09:05 ~ 10:42
Test set: 10:43 ~ 11:25

TEST PREDICTION
Timestamp                 Real            Prediction     
-------------------------------------------------------
2025-04-28 10:43:00+09:00 55600.000000    55598.426376   
2025-04-28 10:44:00+09:00 55600.000000    55598.426376   
2025-04-28 10:45:00+09:00 55600.000000    55598.426376   
2025-04-28 10:46:00+09:00 55650.000000    55598.426376   
2025-04-28 10:47:00+09:00 55600.000000    55642.828665   
2025-04-28 10:48:00+09:00 55700.000000    55578.809552   
2025-04-28 10:49:00+09:00 55600.000000    55670.887991   
2025-04-28 10:

## Large-scale TSP using ACO

In [39]:
!pip install numpy

In [3]:
from math import exp, sin
import numpy as np

class AntColony:
    def __init__(
        self, distances,
        num_ants=20, num_best=10, num_iterations=200,
        alpha=1, beta=2, gamma=0.1,
    ):
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = num_ants
        self.n_best = num_best
        self.n_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for _ in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, self.gamma)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
        return all_time_shortest_path
    
    def spread_pheronome(self, all_paths, n_best, gamma):
        self.pheromone *= (1 - self.gamma)
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, _ in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += exp(-self.distances[move]) / (1 + sin(self.distances[move]) ** 2) 

    def gen_path_dist(self, path):
        total_distance = 0
        for ele in path:
            total_distance += self.distances[ele]
        return total_distance

    def gen_all_paths(self):
        all_paths = []
        for _ in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for _ in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        min_distance = np.inf
        move = None
        for i in self.all_inds:
            if i not in visited and dist[i] < min_distance:
                min_distance = dist[i]
                move = i
        return move

if __name__ == "__main__":
    np.random.seed(1) 
    num_cities = 60
    city_coords = np.random.rand(num_cities, 2) * 1000
    distances = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i == j:
                distances[i, j] = np.inf
            else:
                distances[i, j] = np.linalg.norm(city_coords[i] - city_coords[j])

    ant_colony = AntColony(distances, num_ants=50, num_best=20, num_iterations=200, alpha=1, beta=2, gamma=0.1)
    shortest_path = ant_colony.run()

    city_names = [f"City {i+1}" for i in range(num_cities)]
    city_order = [shortest_path[0][0][0]]
    for move in shortest_path[0]:
        city_order.append(move[1])
    
    print("ANT COLONY OPTIMIZATION")
    print(f"\nShortest Path (corresponding distance: {shortest_path[1]:.2f}):")

    full_order = city_order

    for i in range(0, len(full_order), 10):
        line_cities = full_order[i:i+10]
        if i + 10 >= len(full_order):
            if len(line_cities) > 1:
                line = " -> ".join(city_names[idx] for idx in line_cities[:-1])
                line += " -> " + city_names[line_cities[-1]]
            else:
                line = city_names[line_cities[0]]
            print(line)
        else:
            line = " -> ".join(city_names[idx] for idx in line_cities)
            print(line, end=" ->\n")

ANT COLONY OPTIMIZATION

Shortest Path (corresponding distance: 7559.45):
City 1 -> City 6 -> City 12 -> City 22 -> City 43 -> City 7 -> City 15 -> City 37 -> City 58 -> City 20 ->
City 26 -> City 8 -> City 50 -> City 33 -> City 23 -> City 16 -> City 31 -> City 4 -> City 27 -> City 10 ->
City 56 -> City 3 -> City 14 -> City 25 -> City 38 -> City 48 -> City 29 -> City 28 -> City 60 -> City 18 ->
City 42 -> City 32 -> City 34 -> City 45 -> City 30 -> City 40 -> City 19 -> City 59 -> City 11 -> City 13 ->
City 21 -> City 57 -> City 53 -> City 41 -> City 49 -> City 35 -> City 17 -> City 39 -> City 52 -> City 24 ->
City 36 -> City 47 -> City 9 -> City 5 -> City 51 -> City 2 -> City 54 -> City 46 -> City 44 -> City 55 ->
City 1


In large-scale discrete optimization problems such as the Traveling Salesman Problem (TSP) with fifty or more cities, artificial intelligence (AI) becomes essential due to the infeasibility of exact methods. The number of possible routes grows factorially (O(N!)), making exhaustive search computationally impossible even for moderately sized problems. Simple heuristics often fail to find near-optimal solutions as they easily get trapped in local minima. AI-based metaheuristics like Ant Colony Optimization (ACO) provide adaptive strategies to efficiently explore the solution space. By leveraging collective learning and probabilistic search mechanisms, ACO balances exploration and exploitation, enabling the discovery of high-quality solutions within reasonable time frames. Thus, AI is critical for addressing the scale and complexity that traditional methods cannot handle.